In [1]:
import os
import cv2

from skimage import io
from matplotlib import pyplot as plt
import numpy as np

from skimage import img_as_ubyte, img_as_float

from PIL import Image, ImageOps
from numpy import asarray

from scipy import ndimage as ndi

In [14]:
#Directorio a recorrer
path_filamentos = '1 Frames\Shim_L100_U29-57\\'

def Ficheros(path):
    contenido = os.listdir(path)

    #Acumulador del nombre de las imagenes a utilizar
    imagenes = []

    #Se recorre el path para obtener los nombre y se los aggrega al acumulador, en este caso la lista
    for fichero in contenido:
        if os.path.isfile(os.path.join(path, fichero)) and fichero.endswith('.png'):
            imagenes.append(fichero)
            
    return imagenes

#Se utiliza la función para obtener los fichero que se van a procesar
filamentos = Ficheros(path_filamentos)
print(len(filamentos))

949


In [15]:
#Directorio a guardar los resultados
path_hsv2 = 'HSV--4\Shim_L100_U29-57\\' 

#Se define la función que va a realizar la umbralización
def hsv2(file_name):
    
    pathw = path_hsv2 + file_name
    
    #Se abre el fichero
    img = cv2.imread(path_filamentos + file_name)
    blur = cv2.blur(img,(7,7))
    
    cv2.imwrite(pathw, blur)
    
    gray_image = io.imread(pathw, as_gray=True)
    gray_image = img_as_ubyte(gray_image)

    # convert image to numpy array
    percentil = np.percentile(gray_image, 90)

    p = float(percentil)
    #print(p)
    segm2 = (gray_image > p+2)
    
    all_segments = np.zeros((gray_image.shape[0], gray_image.shape[1], 3), dtype=np.uint8)
    all_segments[segm2] = (255, 255, 255)

    pathw = path_hsv2 + file_name
    plt.imsave(pathw,all_segments)
    
    new_img = cv2.imread(pathw)
    
    #new_img = cv2.rectangle(new_img, (0, 377), (640, 480), (0, 0, 0), -1)
    new_img = cv2.rectangle(new_img, (210, 370), (450, 480), (0, 0, 0), -1)
    new_img = cv2.rectangle(new_img, (590, 410), (640, 480), (0, 0, 0), -1)
    new_img = cv2.rectangle(new_img, (0, 450), (60, 480), (0, 0, 0), -1)
    #new_img = cv2.circle(new_img,(350,292), 20, (0,0,0), -1) 
    #new_img = cv2.circle(new_img,(335,300), 12, (0,0,0), -1)
    
    cv2.imwrite(pathw, new_img)

#Se aplica la función a los ficheros  
for img in range(len(filamentos)):
    hsv2(filamentos[img])

In [16]:
#Directorio a guardar los resultados
path_blob= 'Blob-2\Shim_L100_U29-57\\' 

#Se define la función que va a realizar la operación mencionada
def blob(file_name):
    
    #Se abre el fichero
    img = cv2.imread(path_hsv2 + file_name)
    
    #Se define el kernel con el que se va a realizar la erosion
    kernal = np.ones((5, 5), np.uint8)
    
    #Se aplica la operacion morfológica y sus respectivas iteraciones
    dilation = cv2.dilate(img, kernal, iterations=6)
    
    image = img_as_float(img)
    
    #Se definen los parámetros para realizar la segmentación de areas
    label_objects, nb_labels = ndi.label(image)
    sizes = np.bincount(label_objects.ravel())
    
    mask_sizes = sizes > 600
    mask_sizes[0] = 0
    clean = mask_sizes[label_objects]
    
    clean_2 = img_as_ubyte(clean)
    
    #Se define el kernel con el que se va a realizar la erosion
    kernal = np.ones((3, 3), np.uint8)
    
    #Se aplica la operacion morfológica y sus respectivas iteraciones
    dilation = cv2.dilate(clean_2, kernal, iterations=1)
    
    image = img_as_float(dilation)
    
    #Se definen los parámetros para realizar la segmentación de areas
    label_objects, nb_labels = ndi.label(image)
    sizes = np.bincount(label_objects.ravel())
    
    #Se define las áreas a eliminar
    mask_sizes = sizes > 350
    mask_sizes[0] = 0
    clean = mask_sizes[label_objects]
    
    #El resultado se vuelve a transformar a valor tipo 'uint8'
    clean_2 = img_as_ubyte(clean)
    
    #Se define el kernel con el que se va a realizar la erosion
    kernal = np.ones((5, 5), np.uint8)
    
    #Se aplica la operacion morfológica y sus respectivas iteraciones
    dilation = cv2.dilate(clean_2, kernal, iterations=4)
    
    #Se define el kernel con el que se va a realizar la erosion
    kernal2 = np.ones((7, 7), np.uint8)
    
    #Se aplica la operacion morfológica y sus respectivas iteraciones
    erosion = cv2.erode(dilation, kernal2, iterations=3)
    
    image = img_as_float(erosion)
    
    #Se definen los parámetros para realizar la segmentación de areas
    label_objects, nb_labels = ndi.label(image)
    sizes = np.bincount(label_objects.ravel())
    
    #Se define las áreas a eliminar
    mask_sizes = sizes > 100
    mask_sizes[0] = 0
    clean = mask_sizes[label_objects]
    
    #El resultado se vuelve a transformar a valor tipo 'uint8'
    clean_2 = img_as_ubyte(clean)
        
    #Se define el kernel con el que se va a realizar la erosion
    kernal = np.ones((7, 7), np.uint8)
    
    #Se aplica la operacion morfológica y sus respectivas iteraciones
    dilation = cv2.dilate(clean_2, kernal, iterations=5)
    
    #Se define el kernel con el que se va a realizar la erosion
    kernal2 = np.ones((7, 7), np.uint8)
    
    #Se aplica la operacion morfológica y sus respectivas iteraciones
    erosion = cv2.erode(dilation, kernal2, iterations=3)
    
    erosion = cv2.rectangle(erosion, (145, 460), (200, 480), (0, 0, 0), -1)
    erosion = cv2.rectangle(erosion, (460, 455), (505, 480), (0, 0, 0), -1)
    erosion = cv2.rectangle(erosion, (85, 468), (130, 480), (0, 0, 0), -1)

    #Se guardan resultados
    pathw = path_blob + file_name
    cv2.imwrite(pathw, erosion)

#Se aplica la función a los ficheros  
for img in range(len(filamentos)):
    blob(filamentos[img])

In [17]:
#Extra
#Eliminación de elementos en la parte inferior de la base

#Directorio a guardar los resultados
path_blob= 'Blob-2\Shim_L100_U28-56\\' 

#Se define la función que va a realizar la operación mencionada
def blob(file_name):
    
    pathw = path_blob + file_name
    new_img = cv2.imread(pathw)
    
    new_img = cv2.rectangle(new_img, (0, 450), (640, 480), (0, 0, 0), -1)


    cv2.imwrite(pathw, new_img)

#Se aplica la función a los ficheros  
for img in range(len(filamentos)):
    blob(filamentos[img])